In [202]:
import numpy as np
import pandas as pd
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from nn_tools import Qfirst, Qfollowing, one_epoch_layers,optimiers_list

In [203]:
data=pd.read_csv(f"data/slice{0}.csv", index_col = 0)
data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,166,167,168,169,170,171,172,173,174,175
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.64,0.64,0.68,0.56,0.60,0.60,0.60,0.64,0.52,0.56
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.60,0.68,0.64,0.64,0.64,0.56,0.64,0.60,0.60,0.60
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.56,0.56,0.68,0.64,0.60,0.52,0.52,0.64,0.60,0.56
3,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.52,0.44,0.44,0.52,0.48,0.48,0.44,0.44,0.52,0.48
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.68,0.56,0.52,0.56,0.56,0.64,0.52,0.48,0.52,0.52
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.60,0.60,0.60,0.56,0.56,0.56,0.56,0.56,0.52,0.56
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.56,0.64,0.68,0.48,0.48,0.56,0.60,0.64,0.48,0.48
7,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.72,0.60,0.48,0.56,0.48,0.68,0.56,0.48,0.52,0.48
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.48,0.68,0.60,0.48,0.48,0.48,0.64,0.56,0.48,0.48
9,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.60,0.56,0.52,0.64,0.56,0.56,0.52,0.52,0.60,0.52


In [204]:
def peak_one(data):
    row = data.sample(n=1).iloc[0]
    x1 = row[:52].values
    x2 = row[52:104].values
    c = row[104:].values
    return x1, x2, c

#example
x1, x2, c = peak_one(data)
print("x1:", x1)
print("x2:", x2)
print("c:", c)

x1: [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 0.]
x2: [0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0.]
c: [1.   0.76 0.72 0.68 0.88 0.72 0.72 0.68 0.64 0.84 0.68 0.68 0.64 0.6
 0.8  0.64 0.64 0.6  0.56 0.76 0.6  0.6  0.56 0.56 0.68 0.56 0.56 0.52
 0.52 0.64 0.52 0.52 0.52 0.52 0.6  0.52 1.   0.76 0.72 0.68 0.88 0.72
 0.72 0.68 0.64 0.84 0.68 0.68 0.64 0.6  0.8  0.64 0.64 0.6  0.56 0.76
 0.6  0.6  0.56 0.56 0.68 0.56 0.56 0.52 0.52 0.64 0.52 0.52 0.52 0.52
 0.6  0.52]


# Result


In [205]:
bidding_history=np.zeros(36)
bidding_history[14:17]=1
num_ones = np.count_nonzero(bidding_history == 1)
print(f"Number of 1s in bidding_history: {num_ones}")

Number of 1s in bidding_history: 3


# Algo_p

### Version finale

In [269]:

def perform_bids(x1, x2, scores,history, Q_models, nb_layers):
    num_ones = np.count_nonzero(history == 1)
    bidding_history = history.copy()
    for i in range(num_ones, nb_layers):
        hand = x1 if (i+1) % 2 == 1 else x2  # Détermine la main en fonction du joueur actif
        print("hand",hand)
        state = torch.tensor(hand, dtype=torch.float32) if i == 0 else torch.tensor(np.concatenate([hand, bidding_history]), dtype=torch.float32)
        print("state",state)

        highest_bid = np.max(np.where(bidding_history == 1)) if np.any(bidding_history == 1) else -1

        # passer l'état dans le modèle Q[i] et obtenir les valeurs Q[i]
        with torch.no_grad():
            q_values = Q_models[i](state)  # prédiction du modèle Q[i] pour le state

        q_values_masked = q_values.clone()
        q_values_masked[:highest_bid+1] = -float('inf')
        
        next_a = torch.argmax(q_values_masked).item()  # sélection greedy max(Q)   a mettre la regle next_a>derniere enchere
        print("next_a",next_a)
        bidding_history[next_a] = 1  # met à jour l'historique des enchères

        if next_a == len(bidding_history) - 1 or i == nb_layers - 1:
            last_action = next_a  # sauvegarde de la dernière action 
            last_layer = i
            return scores[last_action], last_layer


In [256]:

Q_models = [Qfirst() if i == 0 else Qfollowing() for i in range(nb_layers)]

perform_bids(x1, x2, c, bidding_history, Q_models, 5)

hand [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0.
 1. 1. 0. 1.]
state tensor([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
next_a 18
hand [0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0.]
state tensor([0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0.,

np.float64(0.6)

In [268]:
def algo_p(action, x1, x2, scores,bidding_history, Q_models, nb_layers):
    updated_bid_history = bidding_history.copy()
    updated_bid_history[action] = 1
    if np.count_nonzero(updated_bid_history) == nb_layers:
        return scores[action]
    final_score,last_layer=perform_bids(x1, x2,scores, updated_bid_history, Q_models, nb_layers)
    return final_score

In [265]:
#example
x1, x2, scores = peak_one(data)
print("x1:", x1)
print("x2:", x2)
print("scores:", scores)
bidding_history=np.zeros(36)
bidding_history[5]=1
action=6

nb_layers = 5
Q_models = [Qfirst() if i == 0 else Qfollowing() for i in range(nb_layers)]

algo_p(action,x1,x2,scores,bidding_history,Q_models,nb_layers)

x1: [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 1. 1.]
x2: [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0.]
scores: [0.72 0.84 0.84 0.76 0.84 0.88 0.84 0.84 0.72 0.84 0.88 0.84 0.84 0.68
 0.76 1.   0.76 0.84 0.6  0.68 0.84 0.84 0.96 0.56 0.64 0.72 0.64 0.68
 0.56 0.6  0.6  0.6  0.64 0.52 0.56 0.56 0.72 0.84 0.84 0.76 0.84 0.88
 0.84 0.84 0.72 0.84 0.88 0.84 0.84 0.68 0.76 1.   0.76 0.84 0.6  0.68
 0.84 0.84 0.96 0.56 0.64 0.72 0.64 0.68 0.56 0.6  0.6  0.6  0.64 0.52
 0.56 0.56]
hand [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 1. 1.]
state tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 1., 1., 0., 0., 1., 0., 1., 0., 0.

np.float64(0.6)

# Algo_E et peak_one

In [194]:
# Stratégie epsilon-greedy pour explorer/exploiter les actions
def algo_e(Q, state, epsilon, all_actions):
    if np.random.rand() < epsilon:
        return np.random.choice(all_actions)
    else:
        with torch.no_grad():
            return np.argmax(Q(state))  

# Sélectionne une ligne aléatoire du dataset et extrait les informations
def peak_one(data):
    row = data.sample(n=1).iloc[0]  # Sélection aléatoire d’une ligne
    x1 = row[:52].values  # Main du joueur 1
    x2 = row[52:104].values  # Main du joueur 2
    scores = row[104:140].values  # Scores des 36 actions possibles joueur
    return x1, x2, scores

In [195]:
def update(D, state, C, j):
    D[j][0] = torch.concat([D[j][0], state.clone().detach().unsqueeze(0)])
    D[j][1] = torch.concat([D[j][1], torch.tensor(C, dtype=torch.float32).unsqueeze(0)])
    return D

# Main 

In [196]:
# Entraînement du modèle spécifique à chaque couche d'enchère
def learning_algo(data,max_iter=100, nb_layers=5, epsilon=0.1,
                  loss= nn.MSELoss(),batch_size=50, device='cpu'):

    D = [[torch.empty(0), torch.empty(0)] for _ in range(nb_layers)]  # Un dataset D par couche
    all_actions = list(range(36))  # 36 actions possibles
    Q_models = [Qfirst() if i == 0 else Qfollowing() for i in range(nb_layers)]
    losses=[]
    for _ in range(max_iter):
        bidding_history = np.zeros(36) # array de 36 pour enregistrer l'historique des enchères
        x1, x2, scores = peak_one(data)

        for j in range(nb_layers):
            C = np.zeros(36)  
            hand = x1 if (j+1) % 2 == 1 else x2  # main du joueur actuel

            # Calcul des coûts des actions
            for i, action in enumerate(all_actions):
                C[i],last_layer = algo_p(action,x1,x2,scores,bidding_history,Q_models,nb_layers)
            
            # state = main si première couche sinon state = main + enchère_précédente
            state = torch.tensor(hand, dtype=torch.float32) if j == 0 else torch.tensor(np.concatenate([hand, bidding_history]), dtype=torch.float32)

            D=update(D,state,C,j) #ajout de l'expérience dans le dataset D_j
            
            next_a = algo_e(Q_models[j], state, epsilon, all_actions)  # Sélection avec Q_j
            
            if next_a == len(bidding_history) - 1:  # si l'action est PASS
                break  
            
            bidding_history[next_a] = 1  # enregistrement de l'action choisie dans l'historique des enchères


        # Entraînement des modèles Q
        opti_list=optimiers_list(Q_models)
        losses.append(one_epoch_layers(Q_models, D, opti_list,loss, batch_size, device))
    return Q_models
    

In [198]:
learning_algo(data,max_iter=2, nb_layers=5, epsilon=0.1,
                  loss= nn.MSELoss(),batch_size=50, device='cpu')

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

### brouillon

In [139]:
torch.tensor(C, dtype=torch.float32).unsqueeze(0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [143]:
D = [[torch.empty(0), torch.empty(0)] for _ in range(5)]
D = update(D, torch.tensor([1, 2, 3], dtype=torch.float32), [0.1, 0.2, 0.3], 0)
D = update(D, torch.tensor([1, 2, 3], dtype=torch.float32), [0.1, 0.2, 0.3], 0)
print(D[0][0])
print(D[0][1])

tensor([[1., 2., 3.],
        [1., 2., 3.]])
tensor([[0.1000, 0.2000, 0.3000],
        [0.1000, 0.2000, 0.3000]])
